# Análisis de resultados
Vamos a analizar los resultados obtenidos con la ayuda de la librería pandas y un poco de teoría de probabilidad


### Importamos las librerías

In [61]:
import pandas as pd
import numpy as np

### Cargamos los datos del archivo csv a un dataframe de pandas e imprimimos las 5 primeras filas

In [62]:
df = pd.read_csv('experimento.csv')
df.head()

,operand_1,operand_2,query_result,server_1_response,server_2_response,server_3_response,unavailable_server
0,2,2,4,True,True,True,None
1,3,3,6,True,True,False,3
2,5,5,10,False,False,True,3
3,2,3,4,False,False,False,None
4,4,4,8,True,True,True,None


### Calculamos el resultado correcto de cada operación o pregunta

In [63]:
df['actual_result'] = df['operand_1'] + df['operand_2']

In [64]:
df.head()

,operand_1,operand_2,query_result,server_1_response,server_2_response,server_3_response,unavailable_server,actual_result
0,2,2,4,True,True,True,None,4
1,3,3,6,True,True,False,3,6
2,5,5,10,False,False,True,3,10
3,2,3,4,False,False,False,None,5
4,4,4,8,True,True,True,None,8


### Determinamos la respuesta correcta que debío enviar el servidor

In [65]:
df['expected_server_response'] = df.apply(lambda row: True if row['query_result'] == row['actual_result'] else False, axis=1)
df

,operand_1,operand_2,query_result,server_1_response,server_2_response,server_3_response,unavailable_server,actual_result,expected_server_response
0,2,2,4,True,True,True,None,4,True
1,3,3,6,True,True,False,3,6,True
2,5,5,10,False,False,True,3,10,True
3,2,3,4,False,False,False,None,5,False
4,4,4,8,True,True,True,None,8,True
5,5,3,9,True,True,False,3,8,False
6,3,4,7,False,True,True,1,7,True
7,9,10,15,False,True,False,2,19,False
8,2,2,5,True,True,True,None,4,False


### Calificamos la veracidad del validador
A continuación vamos a etiquetar cada fila individual de nuestros datos así:

* **true_positive**: Corresponde a aquel caso en el que nuestro validador detectó correctamente cuando un microservicio estaba fallando. Por ejemplo ante la consulta 2 + 2 = 4,  dos de los microservicios contestaron que era correcta, uno contestó que era incorrecta y por consiguiente el validador detectó correctamente que el que contestó diferente estaba fallando. 
* **true_negative**: Corresponde a aquel caso en el que nuestro validador detectó que todos nuestros microservicios estaban funcionando correctamente. Por ejemplo ante la consulta 2 + 2 = 4, los tres microservicios contestaron que era correcta y el validador detectó que ninguno fallaba. 
* **false_positive**: Corresponde a aquel caso en el que nuestro validador detectó que un microservicio estaba fallando y en realidad estaba funcionando correctamente. Por ejemplo ante la consulta 2 + 2 = 4,  dos de los microservicios contestaron que era incorrecta, uno contestó que era correcta y por consiguiente el validador detectó erróneamente que el que contestó diferente estaba fallando. 
* **false_negative**: Corresponde a aquel caso en el que nuestro validador no detectó que un microservicio estaba fallando. Por ejemplo ante la consulta 2 + 2 = 4,  los 3 microservicios contestaron que era incorrecta y por consiguiente el validador detectó erróneamente que no existía falla alguna.

Es de aclarar que se podría argumentar que un falso positivo también implica un falso negativo; por ejemplo, si ante la consulta (2 + 2) == 4, dos microservicios contestan que es incorrecta y uno contesta que es correcta, el validador habrá identificado erróneamente (falso positivo) que el servicio que contestó diferente está fallando cuando no lo está, y a su vez no habría detectado que los que contestaron de forma incorrecta estaban fallando (falsos negativos). Sin embargo, este caso lo vamos a etiquetar como falso positivo para simplificar el análisis.

In [66]:
def label_validator(row):
    server_responses = {
        '1': row['server_1_response'],
        '2': row['server_2_response'],
        '3': row['server_3_response']
    }

    expected_server_response = row['expected_server_response']
    validator_unavailability_resolution = row['unavailable_server']

    if validator_unavailability_resolution == 'None':
        if all(res == expected_server_response for res in server_responses.values()):
            return 'true_negative'
        elif all(res != expected_server_response for res in server_responses.values()):
            return 'false_negative'
    elif server_responses[validator_unavailability_resolution] == expected_server_response:
        return 'false_positive'
    elif server_responses[validator_unavailability_resolution] != expected_server_response:
        return 'true_positive'
    
df['label'] = df.apply(label_validator, axis=1)
df

    

,operand_1,operand_2,query_result,server_1_response,server_2_response,server_3_response,unavailable_server,actual_result,expected_server_response,label
0,2,2,4,True,True,True,None,4,True,true_negative
1,3,3,6,True,True,False,3,6,True,true_positive
2,5,5,10,False,False,True,3,10,True,false_positive
3,2,3,4,False,False,False,None,5,False,true_negative
4,4,4,8,True,True,True,None,8,True,true_negative
5,5,3,9,True,True,False,3,8,False,false_positive
6,3,4,7,False,True,True,1,7,True,true_positive
7,9,10,15,False,True,False,2,19,False,true_positive
8,2,2,5,True,True,True,None,4,False,false_negative
